# <center> Premier League Statistics </center>

The English Premier League is top level english football and it is contested between 20 clubs and operates on a system of promotion and relgation.
Every year millions of fans of the game come together to create imaginary teams  of real life footballers and participate in a game called Fantasy premier league. The fans score points based on their teams actual statistical performance or their perceived contribution on the field of play.


## Index

* [Downloading HTML files](#Downloading-HTML-files)
* [Scraping Data](#Scraping-Data)
* [Data Cleaning](#Data-Cleaning)
* [Updating the CSV](#Updating-the-CSV-file-with-clean-data)
* [Checking for correlation](#Checking-Correlation)
* [Credits](#Credits)
***

## Downloading HTML files

The cell below will not be used right now. Data was extracted this way initially. I currently directly use beautiful soup library to extract data using an URL.

In [18]:
import os
import time
import requests
import sys

stat_list = ["stats","keepers","keepersadv","shooting","passing","passing_types","gca","defense","possession","playingtime","misc"]
def retirieve_pages():
    for stat in stat_list:
        link = "https://fbref.com/en/comps/9/{}/Premier-League-Stats".format(stat)
    
        texts = requests.get(link)
        text_utf = texts.text.encode('utf=8')
    
        if not os.path.exists("/Users/sunsu/Documents/Projects/Football/HTML/{}".format("statistics")):
            os.makedirs("/Users/sunsu/Documents/Projects/Football/HTML/{}".format("statistics"))
        with open("/Users/sunsu/Documents/Projects/Football/HTML/{}/{}.html".format("statistics",stat),"wb")as output:
            output.write(text_utf)
        sys.stdout.flush()
if __name__ == "__main__":
    start_time = time.time()
    retirieve_pages()
    stop_time = time.time()
    print("time taken {}".format(stop_time - start_time))


time taken 8.11665964126587


## Scraping Data

In [27]:
import pandas as pd
import re
import sys, getopt
import csv
from bs4 import BeautifulSoup

def meta_data(stat):
    if stat == "stats":
        html = requests.get("https://fbref.com/en/comps/9/{}/Premier-League-Stats".format(stat),'rb')
        comm = re.compile("<!--|-->")
        soup = BeautifulSoup(comm.sub("",html.text),'lxml')
        all_table = soup.findAll("tbody")
        squad_stats= all_table[0]
        player_stats= all_table[1]
    
#finding player stats
        temp_player= {}
        stats_required = {"player","nationality","position","squad","games","games_starts","minutes","minutes_90s","goals","assists","pens_made","pens_att","cards_yellow","cards_red","goals_per90","assists_per90","goals_assists_per90","goals_pens_per90","goals_assists_pens_per90"}
        rows = player_stats.find_all('tr')
        for row in rows:
            if (row.find("th",{"scope":"row"})!=None):
                for s in stats_required:
                    cell = row.find("td",{"data-stat": s})
                    if cell != None:
                        st = cell.text.strip().encode()
                        text= st.decode("utf-8")
                        if s in temp_player:
                            temp_player[s].append(text)
                        else:
                            temp_player[s] = [text]
        Player_Standard_Stats =  pd.DataFrame.from_dict(temp_player)
        Player_Standard_Stats.to_csv("Player_Standard_Stats.csv")
        
    if stat == "passing":
        html = requests.get("https://fbref.com/en/comps/9/{}/Premier-League-Stats".format(stat),'rb')
        comm = re.compile("<!--|-->")
        soup = BeautifulSoup(comm.sub("",html.text),'lxml')
        all_table = soup.findAll("tbody")
        squad_stats= all_table[0]
        player_stats= all_table[1]
    
#finding player stats
        temp_player= {}
        stats_required = {"player","passes_completed","passes","passes_pct","passes_completed_short","passes_short","passes_pct_short","passes_completed_medium","passes_medium","passes_pct_medium","passes_completed_long","passes_long","cards_yellow","passes_pct_long","assisted_shots","passes_into_final_third","passes_into_penalty_area","crosses_into_penalty_area","progressive_passes"}
        rows = player_stats.find_all('tr')
        for row in rows:
            if (row.find("th",{"scope":"row"})!=None):
                for s in stats_required:
                    cell = row.find("td",{"data-stat": s})
                    if cell != None:
                        st = cell.text.strip().encode()
                        text= st.decode("utf-8")
                        if s in temp_player:
                            temp_player[s].append(text)
                        else:
                            temp_player[s] = [text]
        Player_Passing_Stats =  pd.DataFrame.from_dict(temp_player)
        Player_Passing_Stats.to_csv("Player_Passing_Stats.csv")
        
        
    
    
if __name__ == "__main__":
    start_time = time.time()
    statistics = ["stats","passing"]
    for stats in statistics:
        meta_data(stats)
    stop_time = time.time()
    print("time taken {}".format(stop_time - start_time))

time taken 2.7723724842071533


## Data Cleaning

In [28]:
player_stats = pd.read_csv("Player_Standard_Stats.csv")
passing = pd.read_csv("Player_Passing_Stats.csv")

In [30]:
stats = pd.concat([player_stats,passing],axis = 1)

In [31]:
stats

,Unnamed: 0,minutes,position,cards_red,pens_att,goals_assists_per90,goals_assists_pens_per90,games_starts,player,goals,...,passes_into_penalty_area,player,passes_medium,passes_completed_long,passes_long,passes_completed_short,progressive_passes,assisted_shots,passes_pct_short,passes_pct
0,0,829,DF,0,0,0.11,0.11,9,Patrick van Aanholt,0,...,19,Patrick van Aanholt,206,36,70,233,62,9,87.9,81.1
1,1,898,FW,0,0,0.70,0.70,9,Tammy Abraham,6,...,2,Tammy Abraham,55,9,15,81,10,8,77.1,67.7
2,2,"1,417",FW,0,0,0.51,0.51,17,Che Adams,4,...,11,Che Adams,104,29,41,113,33,18,74.3,69.3
3,3,"1,260",DF,0,0,0.00,0.00,14,Tosin Adarabioyo,0,...,1,Tosin Adarabioyo,322,139,190,95,23,3,88.0,85.7
4,4,180,GK,0,0,0.00,0.00,2,Adrián,0,...,0,Adrián,19,11,29,12,0,0,100.0,69.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468,468,"1,452",MF,0,0,0.19,0.19,16,Andre-Frank Zambo Anguissa,0,...,12,Andre-Frank Zambo Anguissa,345,44,65,324,65,16,87.1,85.0
469,469,64,"FW,DF",0,0,0.00,0.00,0,Andi Zeqiri,0,...,1,Andi Zeqiri,3,1,1,4,1,0,80.0,45.5
470,470,346,DF,0,0,0.00,0.00,3,Oleksandr Zinchenko,0,...,5,Oleksandr Zinchenko,114,22,31,165,25,0,95.9,90.2
471,471,549,FW,0,0,0.66,0.66,7,Hakim Ziyech,1,...,17,Hakim Ziyech,118,32,75,131,35,14,84.0,72.1


In [32]:
stats[["Count","country"]] = stats["nationality"].str.split(" ",expand = True)
stats[["Position_1","Position_2"]] = stats["position"].str.split(",",expand = True)

In [33]:
l = ["Count","Unnamed: 0","nationality","position"]
for ls in l:
    stats = stats.drop([ls],axis =1)

## Updating the CSV file with clean data

In [35]:
stats.to_csv("/Users/sunsu/Documents/Projects/Football/updated_data/Player_complete_Stats.csv")

## Checking Correlation

In [47]:
passing_stats = ['passes_pct_long', 'passes_completed_medium', 'passes_into_final_third',
       'passes', 'passes_completed', 'passes_pct_medium', 'passes_short',
       'crosses_into_penalty_area', 'passes_into_penalty_area','passes_medium', 'passes_completed_long', 'passes_long',
       'passes_completed_short', 'progressive_passes', 'assisted_shots',
       'passes_pct_short', 'passes_pct']
for p in passing_stats:
    print("{} is correlated to assists by {}\n".format(p,(stats[p].corr(stats["assists"])*100)))

passes_pct_long is correlated to assists by -3.384525111418011

passes_completed_medium is correlated to assists by 17.914201099422197

passes_into_final_third is correlated to assists by 31.992537868697628

passes is correlated to assists by 32.205495878628234

passes_completed is correlated to assists by 28.367265014374127

passes_pct_medium is correlated to assists by -11.067710961805249

passes_short is correlated to assists by 45.01276754881423

crosses_into_penalty_area is correlated to assists by 44.126980999729284

passes_into_penalty_area is correlated to assists by 65.47002785740872

passes_medium is correlated to assists by 21.832489876528307

passes_completed_long is correlated to assists by 9.130071029330747

passes_long is correlated to assists by 11.319741198846913

passes_completed_short is correlated to assists by 42.33531591471714

progressive_passes is correlated to assists by 51.368207070043745

assisted_shots is correlated to assists by 75.19843993080683

passes_pc

Assisted shots that is key passes have a very high correlation with assists made. This can be really helpful for players while choosing their teams.

# Credits
Thank you fbref for letting me scrape data